In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'w'
import random
from matplotlib.animation import FuncAnimation
from IPython import display

In [2]:
from time import time
def runtime(t0):
    dt = time() - t0
    if dt < 60:
        message = f'{dt:.0f} sec.'
    elif dt < 3600:
        message = f'{dt/60:.0f} min.'
    else:
        h = int(dt//3600)
        m = int((dt%3600) / 60)
        message = f'{h} h. {m} min.'
    print(message)

In [3]:
class Particle():
    def __init__(self, initial_state=[10, 10, 10, 10], box_size=100):
        self.x  = initial_state[0]
        self.y  = initial_state[1]
        self.vx = initial_state[2]
        self.vy = initial_state[3]
        self.box_size = box_size
        
    def get_position(self):
        return self.x, self.y
    
    def get_velocity(self):
        return self.vx, self.vy
    
    def move(self, dt):
        if (self.x < 0 or self.x > self.box_size):
            # particle at the x boundary
            self.vx = - self.vx
            
        if (self.y < 0 or self.y > self.box_size):
            # particle at the y boundary
            self.vy = - self.vy
            
        self.x += self.vx * dt
        self.y += self.vy * dt

In [4]:
t0 = time()
fig, ax = plt.subplots()
lines = ax.plot([], 'o')
line = lines[0]

# other static stuff
ax.set_aspect('equal')
box_size = 100
ax.set_xlim(0, box_size)
ax.set_ylim(0, box_size)
time_text = ax.text(40, 102, '')

num_particles = 50
particles = []
for _ in range(num_particles):
    x0  = 100*random.uniform(0, 1)
    y0  = 100*random.uniform(0, 1)
    vx0 = 2*random.uniform(-1, 1)
    vy0 = 2*random.uniform(-1, 1)
    initial_state1 = [x0, y0, vx0, vy0]
    p = Particle(initial_state=initial_state1)
    particles.append(p)

def init():
    """initialize animation"""
    line.set_data([], [])
    time_text.set_text('')
        
    return line, time_text

def animate(frame):
    x, y = [], []
    for p in particles:
        p.move(1)
        xy = p.get_position()
        x.append(xy[0])
        y.append(xy[1])
    line.set_data((x, y))
    time_text.set_text(f'time = {frame/2:.0f}')
    return line, time_text

num_loops = 2
    
animt = FuncAnimation(fig, animate, init_func=init, frames=100*num_loops, interval=30, repeat=True)
video = animt.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()
runtime(t0)

9 sec.


In [5]:
animt.save('animt.gif', writer='imagemagick')